In [1]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from load_data import initialize_data
from reading_datasets import read_task5
from labels_to_ids import task5_labels_to_ids
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def train(epoch, train_loader, model, optimizer, device, grad_step = 1, max_grad_norm = 10):
    tr_loss, tr_accuracy = 0, 0
    tr_precision, tr_recall = 0, 0
    tr_f1score = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    optimizer.zero_grad()
    
    for idx, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        if (idx + 1) % 20 == 0:
            print('FINSIHED BATCH:', idx, 'of', len(train_loader))

        #loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output[0]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
        
        # Compute Precision
        tmp_tr_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0 )[2]
        tr_precision += tmp_tr_precision
        
        # Compute Recall
        tmp_tr_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0 )[2]
        tr_recall += tmp_tr_recall
        
        # Compute f1score
        tmp_tr_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(),labels=[0,1,2], average=None, zero_division=0)[2]
        tr_f1score += tmp_tr_f1score
    

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        # backward pass
        output['loss'].backward()
        if (idx + 1) % grad_step == 0:
            optimizer.step()
            optimizer.zero_grad()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    tr_precision = tr_precision / nb_tr_steps
    tr_recall = tr_recall / nb_tr_steps
    tr_f1score= tr_f1score / nb_tr_steps
    #print(f"Training loss epoch: {epoch_loss}")
    #print(f"Training accuracy epoch: {tr_accuracy}")

    return model

In [3]:
def testing(model, testing_loader, labels_to_ids, device):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_precision, eval_recall = 0, 0
    eval_f1score = 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
     
    
    ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            #loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            output = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += output['loss'].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            
            # Compute Precision
            tmp_eval_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0)[2]
            eval_precision += tmp_eval_precision
            
            # Compute Recall
            tmp_eval_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0)[2]
            eval_recall += tmp_eval_recall
            
            # Compute f1score
            tmp_eval_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(),labels=[0,1,2], average=None, zero_division=0)[2]
            eval_f1score += tmp_eval_f1score

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    eval_precision = eval_precision / nb_eval_steps
    eval_recall = eval_recall / nb_eval_steps
    eval_f1score = eval_f1score / nb_eval_steps
    #print(f"Validation Loss: {eval_loss}")
    #print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_accuracy, eval_precision, eval_recall, eval_f1score

In [4]:
def main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location):
    #Initialization training parameters
    max_len = 256
    batch_size = 32
    grad_step = 1
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../2022.07.07_task5/'

    train_data = read_task5(dataset_location , split = 'train')
    dev_data = read_task5(dataset_location , split = 'dev')
    #test_data = read_task5(dataset_location , split = 'dev')#load test set
    labels_to_ids = task5_labels_to_ids
    #input_data = (train_data, dev_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time
    if model_load_flag:
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
    else: 
        tokenizer =  AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)

    #Get dataloaders
    train_loader = initialize_data(tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
    dev_loader = initialize_data(tokenizer, initialization_input, dev_data, labels_to_ids, shuffle = True)
    #test_loader = initialize_data(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = True)#create test loader

    best_dev_acc = 0
    best_test_acc = 0
    best_dev_precision = 0
    best_test_precision = 0
    best_dev_recall = 0
    best_test_recall = 0
    best_dev_f1score = 0
    best_test_f1score = 0
    best_epoch = -1
    
    list_dev_acc = [] 
    list_test_acc = []  
    list_dev_precision = []  
    list_test_precision  = []  
    list_dev_recall = []  
    list_test_recall = []  
    list_dev_f1score = []  
    list_test_f1score = []
    
    for epoch in range(n_epochs):
        start = time.time()
        print(f"Training epoch: {epoch + 1}")

        #train model
        if not model_load_flag:
            model = train(epoch, train_loader, model, optimizer, device, grad_step)
        
        #testing and logging
        labels_dev, predictions_dev, dev_accuracy, dev_precision, dev_recall, dev_f1score = testing(model, dev_loader, labels_to_ids, device)
        print('DEV ACC:', dev_accuracy)
        print('DEV Precision:' , dev_precision)
        print('DEV Recall:' , dev_recall)
        print('DEV F1Score:' , dev_f1score)
        
        list_dev_acc.append(dev_accuracy)     
        list_dev_precision.append(dev_precision)   
        list_dev_recall.append(dev_recall)  
        list_dev_f1score.append(dev_f1score)  
        
        
        #labels_test, predictions_test, test_accuracy, test_precision, test_recall, test_f1score = testing(model, test_loader, labels_to_ids, device)
        #print('TEST ACC:', test_accuracy)
        #print('TEST Precision:' , test_precision)
        #print('TEST Recall:' , test_recall)
        #print('TEST F1Score:' , test_f1score)
        
        #list_test_acc.append(test_accuracy) 
        #list_test_precision.append(test_precision)  
        #list_test_recall.append(test_recall)
        #list_test_f1score.append(test_f1score) 

        #saving model
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy
            #best_test_acc = test_accuracy
        if dev_precision > best_dev_precision:
            best_dev_precision = dev_precision
            #best_test_precision = test_precision
        if dev_recall > best_dev_recall:
            best_dev_recall = dev_recall
            #best_test_recall = test_recall
        if dev_f1score > best_dev_f1score:
            best_dev_f1score = dev_f1score
            #best_test_f1score = test_f1score
            best_epoch = epoch
            
            if model_save_flag:
                os.makedirs(model_save_location, exist_ok=True)
                tokenizer.save_pretrained(model_save_location)
                model.save_pretrained(model_save_location)

        now = time.time()
        print('BEST ACCURACY --> ', 'DEV:', round(best_dev_acc, 5))
        print('BEST PRECISION --> ', 'DEV:', round(best_dev_precision, 5))
        print('BEST RECALL --> ', 'DEV:', round(best_dev_recall, 5))
        print('BEST F1SCORE --> ', 'DEV:', round(best_dev_f1score, 5))
        print('TIME PER EPOCH:', (now-start)/60 )
        print()

    return best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score

In [5]:
if __name__ == '__main__':
    n_epochs = 5
    models = ['bert-base-multilingual-cased']
    
    #model saving parameters
    model_save_flag = True
    model_load_flag = False
    
    overall_list_dev_acc = [] 
    overall_list_test_acc = []    
    overall_list_dev_precision = []  
    overall_list_test_precision  = []  
    overall_list_dev_recall = []  
    overall_list_test_recall = []  
    overall_list_dev_f1score = []  
    overall_list_test_f1score = [] 
    
    for i in range(5):
        
        for model_name in models:

            model_save_location = 'saved_models/' + model_name + str(i)
            model_load_location = None

            best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score = main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location)
            
            overall_list_dev_acc.append(list_dev_acc) 
            overall_list_test_acc.append(list_test_acc) 
            overall_list_dev_precision.append(list_dev_precision)  
            overall_list_test_precision.append(list_test_precision) 
            overall_list_dev_recall.append(list_dev_recall)  
            overall_list_test_recall.append(list_test_recall)  
            overall_list_dev_f1score.append(list_dev_f1score)  
            overall_list_test_f1score.append(list_test_f1score) 

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Training epoch: 1
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHED BATCH: 239 of 252
Validation loss per 100 evaluation steps: 0.20034973323345184
DEV ACC: 0.8236331569664902
DEV Precision: 0.6691403834260978
DEV Recall: 0.8005165028974551
DEV F1Score: 0.7124427001951694
BEST ACCURACY -->  DEV: 0.82363
BEST PRECISION -->  DEV: 0.66914
BEST RECALL -->  DEV: 0.80052
BEST F1SCORE -->  DEV: 0.71244
TIME PER EPOCH: 6.709881325562795

Training epoch: 2
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSI

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Training epoch: 1
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHED BATCH: 239 of 252
Validation loss per 100 evaluation steps: 0.3047519624233246
DEV ACC: 0.8147045855379188
DEV Precision: 0.6191982180077417
DEV Recall: 0.8946694839551981
DEV F1Score: 0.7148121086791288
BEST ACCURACY -->  DEV: 0.8147
BEST PRECISION -->  DEV: 0.6192
BEST RECALL -->  DEV: 0.89467
BEST F1SCORE -->  DEV: 0.71481
TIME PER EPOCH: 6.844246486822764

Training epoch: 2
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHED

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Training epoch: 1
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHED BATCH: 239 of 252
Validation loss per 100 evaluation steps: 0.4000139832496643
DEV ACC: 0.8347295708406819
DEV Precision: 0.7025991204562633
DEV Recall: 0.8053539934492315
DEV F1Score: 0.7340268404694171
BEST ACCURACY -->  DEV: 0.83473
BEST PRECISION -->  DEV: 0.7026
BEST RECALL -->  DEV: 0.80535
BEST F1SCORE -->  DEV: 0.73403
TIME PER EPOCH: 6.788865256309509

Training epoch: 2
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHE

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Training epoch: 1
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHED BATCH: 239 of 252
Validation loss per 100 evaluation steps: 0.3349117636680603
DEV ACC: 0.8180849500293945
DEV Precision: 0.6484018186399139
DEV Recall: 0.8416099773242628
DEV F1Score: 0.7200731236482377
BEST ACCURACY -->  DEV: 0.81808
BEST PRECISION -->  DEV: 0.6484
BEST RECALL -->  DEV: 0.84161
BEST F1SCORE -->  DEV: 0.72007
TIME PER EPOCH: 6.844860637187958

Training epoch: 2
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHE

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Training epoch: 1
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHED BATCH: 239 of 252
Validation loss per 100 evaluation steps: 0.3186711370944977
DEV ACC: 0.8184891240446797
DEV Precision: 0.6243821874774255
DEV Recall: 0.8847860693098786
DEV F1Score: 0.7141067144727187
BEST ACCURACY -->  DEV: 0.81849
BEST PRECISION -->  DEV: 0.62438
BEST RECALL -->  DEV: 0.88479
BEST F1SCORE -->  DEV: 0.71411
TIME PER EPOCH: 6.879524083932241

Training epoch: 2
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIH

In [6]:
print(best_dev_acc)

0.8493349500293945


In [7]:
print(best_epoch)

1


In [8]:
print(best_dev_precision)

0.7215797430083146


In [9]:
print(best_dev_recall)

0.8899092970521542


In [10]:
print(best_dev_f1score)

0.7448576986650108


In [11]:
print(overall_list_dev_acc)

[[0.8236331569664902, 0.8194811875367431, 0.8398184891240447, 0.8401859200470312, 0.8401308054085832], [0.8147045855379188, 0.8278218694885362, 0.8458627278071723, 0.8312022339800118, 0.841086125808348], [0.8347295708406819, 0.8225492357436802, 0.8419863315696648, 0.8422986478542034, 0.8356665196942975], [0.8180849500293945, 0.8365299823633157, 0.8430702527924749, 0.8425191064079953, 0.8222369194591417], [0.8184891240446797, 0.8367504409171076, 0.8491144914756026, 0.8493349500293945, 0.8493349500293945]]


In [12]:
print(overall_list_dev_precision)

[[0.6691403834260978, 0.7099521289997481, 0.7103237591332828, 0.6895920657825421, 0.6898686410591176], [0.6191982180077417, 0.7446082136558327, 0.6786911060720585, 0.663394718156623, 0.6492379313807886], [0.7025991204562633, 0.6329598531979487, 0.7139438603724317, 0.6624040862136099, 0.6605321926750498], [0.6484018186399139, 0.6690361666552144, 0.6888001328477519, 0.7087238599143358, 0.6681058800106421], [0.6243821874774255, 0.6649201768249386, 0.6907804717328526, 0.7215797430083146, 0.658332760713713]]


In [13]:
print(overall_list_dev_recall)

[[0.8005165028974551, 0.5489103048626857, 0.7355568153187202, 0.7699105568153187, 0.7784706475182666], [0.8946694839551981, 0.6223293020912071, 0.8079805996472663, 0.8277955289860051, 0.9022234819853866], [0.8053539934492315, 0.9126417233560091, 0.6653088534040915, 0.8693625598387502, 0.7997480473670948], [0.8416099773242628, 0.84937641723356, 0.8306248425296043, 0.7616276140085663, 0.8558167141500472], [0.8847860693098786, 0.8899092970521542, 0.7078042328042327, 0.6729488765203053, 0.8786542822257106]]


In [14]:
print(overall_list_dev_f1score)

[[0.7124427001951694, 0.601333322761894, 0.7057421906861681, 0.7127568740387549, 0.708495612697293], [0.7148121086791288, 0.6592020413448986, 0.7232984944137825, 0.7188054398138431, 0.7438834187691623], [0.7340268404694171, 0.7319314151379928, 0.6690647597370926, 0.7402936333682314, 0.7052597690884583], [0.7200731236482377, 0.7340460216584346, 0.7371211682099192, 0.7122376790517728, 0.7331291681601275], [0.7141067144727187, 0.7448576986650108, 0.6859711976503954, 0.6692764492484379, 0.7375744939218839]]


In [15]:
#Best model is model 4